Импортируем необходимые библиотеки

In [1]:
import pandahouse as ph
import pandas as pd

Объявляем подключения к базе данных

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Этап загрузки всех данных из таблиц и их разведывательный анализ не целесообразен ввиду большого объема таблиц. В качестве предварительного анализа стоит произвести несколько направленных SQL-запросов с целью отследить необходимые параметры для дальнейшей работы

### 1. Общий взгляд на таблицы и что в них содержится

In [3]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query_installs = """
SELECT *
FROM default.installs
LIMIT 10
"""
installs = ph.read_clickhouse(query_installs, connection=connection)

print('Таблица installs\n', installs) 

query_events = """
SELECT *
FROM default.events
LIMIT 10
"""
events = ph.read_clickhouse(query_events, connection=connection)
print('\nТаблица events\n', events) 

Таблица installs
   InstallationDate  InstallCost Platform              DeviceID     Source
0       2019-03-02            0  android   7950068545577019282  Source_27
1       2019-03-17           49  android  17173992779193729517  Source_14
2       2019-04-07           56  android   9528182466778893591  Source_14
3       2019-06-25           39  android   2212531864415574595   Source_9
4       2019-04-13            0  android   6959033924999748551  Source_27
5       2019-01-27            0  android    262831700047781552  Source_27
6       2019-07-16            0  android   8026675461764302030  Source_27
7       2019-09-19            0  android  14556943550242489445  Source_27
8       2019-03-31            0      iOS   2367522355246575108  Source_27
9       2019-02-14            0      iOS   7134503170474894060  Source_27

Таблица events
   AppPlatform  events   EventDate             DeviceID
0     android       8  2019-09-29  7429291373250434008
1     android     175  2019-09-15  742929

In [4]:
query_checks = """
SELECT *
FROM default.checks
LIMIT 10
"""
checks = ph.read_clickhouse(query_checks, connection=connection)
print('Таблица checks\n', checks) 

query_devices = """
SELECT *
FROM default.devices
LIMIT 10
"""
devices = ph.read_clickhouse(query_devices, connection=connection)
print('\nТаблица devices\n', devices) 

Таблица checks
    Rub     BuyDate                UserID
0    3  2019-10-04  18446583642950580515
1    4  2019-10-04  18446535622689003675
2    0  2019-10-04  18446130411954852964
3    3  2019-10-04  18446003252714243011
4    8  2019-10-04  18445948434655311802
5   17  2019-10-04  18445927732647659917
6    2  2019-10-04  18445761122620052505
7   15  2019-10-04  18445655133428855896
8    1  2019-10-04  18445587876544434519
9    1  2019-10-04  18445586451093345117

Таблица devices
          DeviceID                UserID
0    290132773793   9407952136059258036
1    619578718457  14561372283986042425
2   9011245721293    284698082657182871
3  11452758091056  17260156507956968854
4  12397387173463   4870696193129182034
5  12599755717733   6758018588637972435
6  13210761921520   3955806493898881684
7  15216677602256  15186450329800833643
8  16775400460803   5591788874343608413
9  21701379700142  14176532582507529800


Посмотрим, за какой период мы имеем данные по каждой из таблиц

In [5]:
query_installs = """
SELECT  min(InstallationDate) AS MinDate,
        max(InstallationDate) as MaxDate 
FROM installs
"""
installs = ph.read_clickhouse(query_installs, connection=connection)

print('Таблица installs\n', installs) 

query_events = """
SELECT  min(EventDate) AS MinDate,
        max(EventDate) as MaxDate 
FROM events
"""
events = ph.read_clickhouse(query_events, connection=connection)
print('\nТаблица events\n', events) 

query_checks = """
SELECT  min(BuyDate) AS MinDate,
        max(BuyDate) as MaxDate 
FROM checks
"""
checks = ph.read_clickhouse(query_checks, connection=connection)
print('\nТаблица checks\n', checks) 

Таблица installs
       MinDate     MaxDate
0  2019-01-01  2019-10-04

Таблица events
       MinDate     MaxDate
0  2019-01-01  2019-10-04

Таблица checks
       MinDate     MaxDate
0  2019-01-01  2019-10-04


Период во всех таблицах совпадает. 

Начнем исследование. В качестве первого шага предлагаю посмотреть количество сделанных покупок и сумму потраченных денег с разбивкой по клиентам (топ-100) и сортировкой по выручке по убыванию:

In [6]:
query_checks = """
SELECT UserID,
        COUNT(*) AS NumChecks,
        SUM(Rub) AS Revenue
FROM checks
GROUP BY UserID
ORDER BY Revenue DESC
LIMIT 100
"""
checks = ph.read_clickhouse(query_checks, connection=connection)
print('\nТаблица checks\n', checks) 


Таблица checks
                   UserID  NumChecks  Revenue
0   15605251414578189718          3   181146
1   16967914029594691558         10   139992
2    9613023063681218234         51    70128
3   11863769084622496656         32    63193
4     958354669642668056         49    60196
..                   ...        ...      ...
95  17041548600936598838        104    18937
96  16413894160611333042         30    18769
97   8296588862270765184         21    18745
98    393664612666139677         50    18717
99   6154751422120351802        175    18592

[100 rows x 3 columns]


Просчитаем по дням минимальный, средний, и максимальный чек:

In [7]:
query_checks = """
SELECT  BuyDate,
        MIN(Rub) AS MinCheck,
        MAX(Rub) AS MaxCheck,
        AVG(Rub) AS AvgCheck
FROM checks
GROUP BY BuyDate
ORDER BY BuyDate DESC
LIMIT 100
"""
checks = ph.read_clickhouse(query_checks, connection=connection)
print('\nТаблица checks\n', checks) 


Таблица checks
        BuyDate  MinCheck  MaxCheck   AvgCheck
0   2019-10-04         0      3239  11.054391
1   2019-10-03         0      4898  10.357468
2   2019-10-02         0      3652   9.743485
3   2019-10-01         0      2928   9.813259
4   2019-09-30         0      4189   9.755777
..         ...       ...       ...        ...
95  2019-07-01         0     15702   9.694462
96  2019-06-30         0      8911  11.492306
97  2019-06-29         0     26422  12.767761
98  2019-06-28         0      4742  11.634987
99  2019-06-27         0     12076  10.657850

[100 rows x 4 columns]


Теперь посмотрим на данные по установкам. Добавим к таблице данные по ID пользователей

In [8]:
query_installs = """
SELECT l.*,
        r.UserID AS UserID
FROM installs AS l
JOIN devices AS r  
    ON l.DeviceID = r.DeviceID
ORDER BY DeviceID DESC
LIMIT 100
"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,InstallationDate,InstallCost,Platform,DeviceID,Source,UserID
0,2019-01-18,266,android,18446743560964479140,Source_9,8158889386484182099
1,2019-04-27,0,android,18446737761282090431,Source_27,14118350678968556622
2,2019-08-27,0,android,18446734879144012952,Source_27,2371786700958450081
3,2019-03-10,0,iOS,18446734469338744354,Source_27,1682752724622218969
4,2019-03-18,0,android,18446727354723047613,Source_27,4112509213117787063
...,...,...,...,...,...,...
95,2019-04-04,85,iOS,18446415452498964691,Source_9,17705336511458439757
96,2019-05-28,0,android,18446413621587257196,Source_27,5126960433734907365
97,2019-02-22,0,android,18446405724585390557,Source_27,8682618813700098111
98,2019-03-26,288,android,18446405021706585563,Source_14,15730467119719812800


Найдем лидера среди источников пользователей

In [10]:
query_installs = """
SELECT i.Source,
        COUNT(DISTINCT c.UserID) AS num_purchases
FROM installs AS i
JOIN devices AS d  
    ON i.DeviceID = d.DeviceID
JOIN checks AS c 
    ON c.UserID = d.UserID
GROUP BY i.Source
ORDER BY num_purchases DESC
LIMIT 3
"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,i.Source,num_purchases
0,Source_27,1848364
1,Source_9,1090975
2,Source_14,542816


Теперь выясним, сколько всего уникальных юзеров совершили покупки в нашем приложении

In [12]:
query_installs = """
SELECT COUNT(DISTINCT c.UserID) AS num_unique_users,
        i.Source
FROM installs AS i
JOIN devices AS d ON i.DeviceID = d.DeviceID
JOIN checks AS c ON d.UserID = c.UserID
GROUP BY i.Source
ORDER BY num_unique_users DESC
"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,num_unique_users,i.Source
0,1848364,Source_27
1,1090975,Source_9
2,542816,Source_14
3,40474,Source_15
4,23316,Source_6
5,17477,Source_5
6,11432,Source_7
7,5276,Source_18
8,2558,Source_26
9,2353,Source_11


Рассчитаем общую выручку, а также минимальный, максимальный и средний чек в разбивке по источникам прихода пользователей

In [13]:
query_installs = """
SELECT COUNT(DISTINCT c.UserID) AS num_unique_users,
        i.Source,
        SUM(c.Rub) AS total_revenue,
        MIN(c.Rub) AS min_check,
        MAX(c.Rub) AS max_check,
        AVG(c.Rub) AS avg_check
FROM installs AS i
JOIN devices AS d ON i.DeviceID = d.DeviceID
JOIN checks AS c ON d.UserID = c.UserID
GROUP BY i.Source
"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,num_unique_users,i.Source,total_revenue,min_check,max_check,avg_check
0,5,Source_1,74,0,19,2.242424
1,783,Source_19,87842,0,484,11.119241
2,2,Source_10,57,0,10,3.166667
3,220,Source_2,4317,0,98,7.708929
4,4,Source_20,77,1,36,7.700000
5,11432,Source_7,1228577,0,4463,10.662880
6,542816,Source_14,33655847,0,38189,9.910412
7,1031,Source_24,97458,0,1241,10.558830
8,2353,Source_11,195917,0,1390,10.009554
9,5276,Source_18,508285,0,1675,10.133476


Проверим, сколько товаров в среднем просматривают пользователи с разных платформ, и пришедшие из разных источников:

In [14]:
query_installs = """
SELECT
    i.Platform,
    i.Source,
    AVG(e.events) AS avg_views
FROM
    installs AS i
JOIN
    events AS e 
    ON i.DeviceID = e.DeviceID AND i.Platform = e.AppPlatform
GROUP BY
    i.Platform,
    i.Source
ORDER BY
    avg_views DESC
"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,Platform,Source,avg_views
0,iOS,Source_20,124.000000
1,android,Source_16,42.945455
2,android,Source_21,35.132530
3,android,Source_17,34.132075
4,android,Source_2,32.423400
5,android,Source_23,31.260719
6,iOS,Source_8,30.809524
7,android,Source_8,30.673566
8,android,Source_22,30.415493
9,iOS,Source_7,29.520909


Посчитаем конверсию в просмотр из установок на платформе iOS:

In [19]:
query_installs = """
SELECT uniqExact(e.DeviceID)  / uniqExact(i.DeviceID)
FROM installs as i
LEFT JOIN events as e using(DeviceID)

"""
installs = ph.read_clickhouse(query_installs, connection=connection)
installs

,"divide(uniqExact(e.DeviceID), uniqExact(DeviceID))"
0,0.986646
